In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import strip_tags
from gensim.models.doc2vec import TaggedDocument
from lbl2vec import Lbl2Vec
from sklearn.metrics import f1_score

In [8]:
labels = pd.DataFrame({'name' : ['facturen','aanmaningen'], 'classIndex' : [1,2], 'keywords' : [['rekening','afrekening','factuur'],['aanmaning','herinnering']]})

In [9]:
fullDataset = pd.read_csv(os.getcwd() + "\\test_extraction_emails.csv")
trainSet, testSet = train_test_split(fullDataset, test_size=0.2)

trainSet['type'] = 'train'
testSet['type'] = 'test'
fullCorpus = pd.concat([trainSet, testSet]).reset_index(drop=True)
fullCorpus = fullCorpus[fullCorpus['classIndex'].isin(list(labels['classIndex']))]

def tokenize(doc):
    return simple_preprocess(strip_tags(doc), deacc=True, min_len=2, max_len=15)
fullCorpus = fullCorpus.dropna()
fullCorpus['tagged'] = fullCorpus.apply(lambda row: TaggedDocument(tokenize(row['contents']), [str(row.name)]), axis=1)
fullCorpus['documentKey'] = fullCorpus.index.astype(str)
fullCorpus = fullCorpus.merge(labels, left_on='classIndex', right_on='classIndex', how='left')
fullCorpus.head()

,Unnamed: 0,names,contents,classIndex,type,tagged,documentKey,name,keywords
0,1184,a30c3018-73a2-48c8-ad4f-f120aa8c6efb-facturen,Message from Basware Invoice from InetumRealdo...,1,train,"([message, from, basware, invoice, from, sent,...",0,facturen,"[rekening, afrekening, factuur]"
1,1707,ed963fe9-374e-4eb8-9e7e-30538bf54e11-facturen,"Dag Mireille, Onderstaande betaling zou moeten...",1,train,"([dag, mireille, onderstaande, betaling, zou, ...",1,facturen,"[rekening, afrekening, factuur]"
2,527,48a461f3-f183-4874-babe-3cc0867af2b3-facturen,"Hi , Please find enclosed invoices Thanks Reg...",1,train,"([hi, please, find, enclosed, invoices, thanks...",2,facturen,"[rekening, afrekening, factuur]"
3,1388,c24d3657-b5df-4df9-965f-71c6b3b74e34-facturen,"!if !vml!endifUw factuurGeachte, Beste,In bijl...",1,train,"([if, vml, endifuw, factuurgeachte, beste, in,...",3,facturen,"[rekening, afrekening, factuur]"
4,27,02a31090-cecb-4835-a601-7a07507194f9-facturen,Deze factuur graag nog even on hold zetten. Zo...,1,train,"([deze, factuur, graag, nog, even, on, hold, z...",4,facturen,"[rekening, afrekening, factuur]"


In [22]:
Lbl2VecModel = Lbl2Vec(keywords_list=list(labels.keywords), tagged_documents=fullCorpus['tagged'][fullCorpus['type'] == 'train'], label_names=list(labels.name), min_count=2, similarity_threshold=0.43, min_num_docs=10, epochs=10)

Lbl2VecModel.fit()

2023-02-15 17:00:35,182 - Lbl2Vec - INFO - Train document and word embeddings
2023-02-15 17:00:35,182 - Lbl2Vec - INFO - Train document and word embeddings
2023-02-15 17:00:35,182 - Lbl2Vec - INFO - Train document and word embeddings
2023-02-15 17:00:35,182 - Lbl2Vec - INFO - Train document and word embeddings
2023-02-15 17:00:35,182 - Lbl2Vec - INFO - Train document and word embeddings
2023-02-15 17:00:37,960 - Lbl2Vec - INFO - Train label embeddings
2023-02-15 17:00:37,960 - Lbl2Vec - INFO - Train label embeddings
2023-02-15 17:00:37,960 - Lbl2Vec - INFO - Train label embeddings
2023-02-15 17:00:37,960 - Lbl2Vec - INFO - Train label embeddings
2023-02-15 17:00:37,960 - Lbl2Vec - INFO - Train label embeddings
2023-02-15 17:00:37,962 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: afrekening
2023-02-15 17:00:37,962 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are u

In [23]:
model_docs_lbl_similarities = Lbl2VecModel.predict_model_docs()
print(model_docs_lbl_similarities.columns)
evaluationTrain = model_docs_lbl_similarities.merge(fullCorpus[fullCorpus['type'] == 'train'], left_on='doc_key', right_on='documentKey')
yTrueTrain = evaluationTrain['name']
yPredTrain = evaluationTrain['most_similar_label']

print('F1 score:',f1_score(yTrueTrain, yPredTrain, average='micro'))

2023-02-15 17:00:44,539 - Lbl2Vec - INFO - Get document embeddings from model
2023-02-15 17:00:44,539 - Lbl2Vec - INFO - Get document embeddings from model
2023-02-15 17:00:44,539 - Lbl2Vec - INFO - Get document embeddings from model
2023-02-15 17:00:44,539 - Lbl2Vec - INFO - Get document embeddings from model
2023-02-15 17:00:44,539 - Lbl2Vec - INFO - Get document embeddings from model
2023-02-15 17:00:44,543 - Lbl2Vec - INFO - Calculate document<->label similarities
2023-02-15 17:00:44,543 - Lbl2Vec - INFO - Calculate document<->label similarities
2023-02-15 17:00:44,543 - Lbl2Vec - INFO - Calculate document<->label similarities
2023-02-15 17:00:44,543 - Lbl2Vec - INFO - Calculate document<->label similarities
2023-02-15 17:00:44,543 - Lbl2Vec - INFO - Calculate document<->label similarities


Index(['doc_key', 'most_similar_label', 'highest_similarity_score', 'facturen',
       'aanmaningen'],
      dtype='object')
F1 score: 0.39057239057239057


In [24]:
new_docs_lbl_similarities = Lbl2VecModel.predict_new_docs(tagged_docs=fullCorpus['tagged'][fullCorpus['type']=='test'])

# merge DataFrames to compare the predicted and true topic labels
evaluationTest = new_docs_lbl_similarities.merge(fullCorpus[fullCorpus['type']=='test'], left_on='doc_key', right_on='documentKey')
yTrueTest = evaluationTest['name']
yPredTest = evaluationTest['most_similar_label']

print('F1 score:',f1_score(yTrueTest, yPredTest, average='micro'))

2023-02-15 17:00:48,825 - Lbl2Vec - INFO - Calculate document embeddings
2023-02-15 17:00:48,825 - Lbl2Vec - INFO - Calculate document embeddings
2023-02-15 17:00:48,825 - Lbl2Vec - INFO - Calculate document embeddings
2023-02-15 17:00:48,825 - Lbl2Vec - INFO - Calculate document embeddings
2023-02-15 17:00:48,825 - Lbl2Vec - INFO - Calculate document embeddings
2023-02-15 17:00:49,050 - Lbl2Vec - INFO - Calculate document<->label similarities
2023-02-15 17:00:49,050 - Lbl2Vec - INFO - Calculate document<->label similarities
2023-02-15 17:00:49,050 - Lbl2Vec - INFO - Calculate document<->label similarities
2023-02-15 17:00:49,050 - Lbl2Vec - INFO - Calculate document<->label similarities
2023-02-15 17:00:49,050 - Lbl2Vec - INFO - Calculate document<->label similarities


F1 score: 0.40816326530612246


#### Sources:
- https://towardsdatascience.com/unsupervised-text-classification-with-lbl2vec-6c5e040354de